In [147]:
import cv2
import numpy as np
import mediapipe as mp
import pickle
import os
import time
import tensorflow as tf
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.regularizers import l2
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

In [148]:
#comunicacion y camara
import socket
import threading

In [149]:
#voz
import pyttsx3 
import threading

In [150]:
# Inicializa el motor de texto-a-voz de pyttsx3
tts_engine = pyttsx3.init()  # Crea una instancia del motor TTS

tts_engine.setProperty('rate', 150)  # Velocidad del habla (opcional)

# Crea un objeto de bloqueo para sincronización de hilos
tts_lock = threading.Lock()  # Previene acceso concurrente al motor TTS

# Variable global para almacenar la última seña vocalizada
last_spoken_gesture = None  # Guarda el texto del último gesto reproducido

#lógica de prevención de repeticiones 
def speak_text(text):
    global last_spoken_gesture  # Accede a la variable global
    
    # Bloquea el acceso concurrente usando with para manejo seguro del recurso
    with tts_lock:  # Asegura que solo un hilo use el motor TTS a la vez
        
        # Verifica si el texto es diferente al último reproducido
        if text != last_spoken_gesture:  # Evita repeticiones consecutivas
            
            # Actualiza el registro del último gesto vocalizado
            last_spoken_gesture = text  # Almacena el nuevo texto
            
            # Añade el texto a la cola de reproducción
            tts_engine.say(text)  # Programa la reproducción del texto
            
            # Ejecuta la reproducción y espera a que termine
            tts_engine.runAndWait()  # Bloquea hasta terminar la reproducción

In [151]:
class UDPCamera:
    def __init__(self):
        self.host = '0.0.0.0'
        self.port = 5000
        self.buffer_size = 65536
        self.mtu = 1400  # Tamaño máximo de fragmento
        self.sock = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
        self.sock.settimeout(2)
        self.frame = None
        self.fragments = []  # Almacena fragmentos del frame
        self.running = False
        self.thread = None
        self.lock = threading.Lock()  # Para acceso thread-safe
        self.start()

    def start(self):
        if not self.running:
            self.running = True
            self.sock.bind((self.host, self.port))
            
            self.thread = threading.Thread(target=self._receive_frames, daemon=True)
            self.thread.start()

    def _receive_frames(self):
        while self.running:
            try:
                # Recibir fragmento
                fragment, _ = self.sock.recvfrom(self.buffer_size)
                
                with self.lock:
                    self.fragments.append(fragment)

                    # Detectar último fragmento (tamaño < MTU)
                    if len(fragment) < self.mtu:
                        # Reconstruir frame completo
                        frame_bytes = b''.join(self.fragments)
                        self.fragments = []  # Resetear fragmentos

                        # Decodificar y almacenar frame
                        frame_array = np.frombuffer(frame_bytes, dtype=np.uint8)
                        self.frame = cv2.imdecode(frame_array, cv2.IMREAD_COLOR)

            except socket.timeout:
                print("[WARN] Timeout esperando fragmentos UDP")
                continue
            except Exception as e:
                print(f"[ERROR] Recepción UDP: {str(e)}")
                break

    def read(self):
        with self.lock:
            if self.frame is not None:
                return True, self.frame.copy()  # Copia para thread-safe
            return False, None

    def isOpened(self):
        return self.running

    def release(self):
        self.running = False
        with self.lock:
            self.fragments = []
            self.frame = None
        if self.thread and self.thread.is_alive():
            self.thread.join(timeout=1)
        self.sock.close()
        print("Cámara UDP liberada")

    def __del__(self):
        self.release()

In [152]:
class TFLiteModel:
    def __init__(self, model_path):
        # Cargar el modelo TFLite
        self.interpreter = tf.lite.Interpreter(model_path=model_path)
        self.interpreter.allocate_tensors()
        
        # Obtener detalles de entrada y salida
        self.input_details = self.interpreter.get_input_details()
        self.output_details = self.interpreter.get_output_details()
    
    def predict(self, input_data):
        # Asegurar el tipo de dato correcto y agregar dimensión batch si es necesario
        input_data = np.array(input_data, dtype=self.input_details[0]['dtype'])
        if len(input_data.shape) == len(self.input_details[0]['shape']) - 1:
            input_data = np.expand_dims(input_data, axis=0)
        
        # Establecer la entrada y ejecutar la inferencia
        self.interpreter.set_tensor(self.input_details[0]['index'], input_data)
        self.interpreter.invoke()
        
        # Obtener la salida
        output_data = self.interpreter.get_tensor(self.output_details[0]['index'])
        return output_data


In [153]:
# Configuración de TensorFlow para rendimiento
physical_devices = tf.config.list_physical_devices('GPU')
if physical_devices:
    # Configuración de TensorFlow para rendimiento en CPU
    try:
        # Verificar si hay GPU disponible (para futuras expansiones)
        physical_devices = tf.config.list_physical_devices('GPU')
        
        if physical_devices:
            # Configuración para GPU (no se ejecutará en tu caso)
            for device in physical_devices:
                tf.config.experimental.set_memory_growth(device, True)
            print("GPU disponible para aceleración")
        else:
            # Optimización para CPU
            tf.config.threading.set_intra_op_parallelism_threads(4)  # Aprovecha núcleos físicos
            tf.config.threading.set_inter_op_parallelism_threads(2)  # Paralelismo entre operaciones
            print("Modo CPU activado: Configuración optimizada para Intel Core i7-7500U")
            
    except Exception as e:
        print(f"Error de configuración: {str(e)}")
        print("Usando configuración por defecto de CPU")


In [154]:
# Inicializar MediaPipe
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(
    static_image_mode=False,
    max_num_hands=2,
    min_detection_confidence=0.5, #probar con 0.4
    min_tracking_confidence=0.5 #probar con 0.4
)
mp_drawing = mp.solutions.drawing_utils


In [155]:
# Configuración de directorios y archivos
data_dir = "hand_gestures_data_v13_2"
os.makedirs(data_dir, exist_ok=True)

# Modelo y datos de entrenamiento
model = None
# Inicializar scaler y label encoder
scaler = StandardScaler()
label_encoder = LabelEncoder()
model_file = "hand_gesture_nn_model_v13_2.h5"
scaler_file = "hand_gesture_scaler_v13_2.pkl"
encoder_file = "hand_gesture_encoder_v13_2.pkl"
gesture_data = "gesture_data_v13_2.pkl" 
model_tflite = "modelo_optimizadotl_v13_2.tflite"

Aquí tienes la explicación detestructura de los archivos clave del sistema:

### 1. **hand_gestures_data_4_3/** (Directorio)
- **Qué es**: Carpeta que almacena datos crudos de entrenamiento.
- **Contenido**:
  - Archivos `.pkl` con pares de:
  - `features`: Lista de landmarks (126 valores numéricos por muestra).
  - `labels`: Nombres de gestos asociados (ej: "Hola", "Chau").
- **Función**: Base de datos para entrenar/reentrenar el modelo.

---

### 2. **hand_gesture_nn_model_4_3.h5** (Archivo de modelo)
- **Qué es**: Modelo de red neuronal entrenado en formato HDF5.
- **Contenido**:
  - Arquitectura de la red (capas y conexiones).
  - Pesos aprendidos durante el entrenamiento.
  - Hiperparámetros de configuración.
- **Función**: Realiza predicciones usando los landmarks detectados.

---

### 3. **hand_gesture_scaler_4_3.pkl** (Archivo de preprocesamiento)
- **Qué es**: Objeto `StandardScaler` serializado.
- **Contenido**:
  - Media y desviación estándar de cada característica.
  ```python
  Ejemplo de datos almacenados:
  mean = [0.12, 0.45, ..., 0.78]  # 126 valores
  std = [0.02, 0.15, ..., 0.23]    # 126 valores
  ```
- **Función**: Normaliza nuevos datos usando los mismos parámetros del entrenamiento.

---

### 4. **hand_gesture_encoder_4_3.pkl** (Archivo de codificación)
- **Qué es**: Objeto `LabelEncoder` serializado.
- **Contenido**:
  - Mapeo entre nombres de gestos y números:
  ```python
  Ejemplo:
  {"Hola": 0, "Chau": 1, "Buenos días": 2}
  ```
- **Función**: Convierte:  
  `Predicción numérica → Nombre de gesto legible`.

---

### 5. **gesture_data_4_3.pkl** (Datos procesados)
- **Qué es**: Versión serializada de los datos listos para entrenamiento.
- **Contenido**:
  - Datos ya normalizados (`X_scaled`).
  - Etiquetas codificadas numéricamente (`y_encoded`).
- **Función**: Acelera el reentrenamiento al evitar reprocesar datos crudos.

---

### Flujo de trabajo:
1. **Recolección**:  
   Landmarks crudos → Se guardan en `hand_gestures_data_4_3/`.

2. **Entrenamiento**:  
   Usa `scaler` para normalizar y `encoder` para codificar → Genera el modelo `.h5`.

3. **Predicción**:  
   Nuevos landmarks → Se normalizan con `scaler.pkl` → Modelo `.h5` predice → `encoder.pkl` traduce a texto.

Estos archivos forman un ecosistema completo que permite actualizar gestos, reentrenar el modelo, y mantener consistencia en las predicciones.

In [156]:
# Variables globales para estado
data = []
labels = []

# Estado del sistema
is_trained = False
is_collecting = False
current_gesture = ""
samples_collected = 0
max_samples = 100

# Control de tiempo para la recolección continua
last_sample_time = 0
sample_delay = 0.05  # 50ms entre muestras

# Temporizador para mostrar mensajes
message = ""
message_until = 0

# Para evaluación del modelo
metrics = {
    'accuracy': 0,
    'val_accuracy': 0,
    'training_time': 0
}

In [157]:
def extract_hand_landmarks(frame):
    """
    Extrae los landmarks de las manos desde un frame de video.

    Args:
        frame: Imagen capturada por la cámara (en formato BGR).

    Returns:
        tuple: Lista de landmarks normalizados (126 elementos) y booleano indicando si se detectaron manos.
    """
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = hands.process(frame_rgb)
    
    landmarks_data = []
    hands_detected = False
    
    if results.multi_hand_landmarks:
        hands_detected = True
        # Extraer landmarks de hasta dos manos
        for hand_landmarks in results.multi_hand_landmarks:
            # Dibujar landmarks en el frame
            mp_drawing.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)
            landmarks = []

            # Extraer coordenadas (x,y,z) de los 21 landmarks
            for landmark in hand_landmarks.landmark:
                landmarks.extend([landmark.x, landmark.y, landmark.z])

            landmarks_data.extend(landmarks)
    
    # Normalizar para 2 manos (si solo hay una o ninguna, rellenar con ceros)
    while len(landmarks_data) < 21 * 3 * 2:  # 21 puntos * 3 coordenadas * 2 manos
        landmarks_data.append(0.0)
    
    # Limitar a exactamente 126 características (21 puntos * 3 coordenadas * 2 manos)
    landmarks_data = landmarks_data[:21 * 3 * 2]
    
    return landmarks_data, hands_detected

In [158]:
def set_message(message_text, duration=2):
    """
    Establece un mensaje para mostrar en pantalla por una duración determinada.

    Args:
        message_text: Mensaje a mostrar (str).
        duration: Duración en segundos (int o float, por defecto 2).
    """
    global message, message_until
    message = message_text
    message_until = time.time() + duration

In [159]:
def start_collection(gesture_name):
    """
    Inicia la recolección de datos para una seña específica.

    Args:
        gesture_name: Nombre de la seña a recolectar (str).
    """
    global is_collecting, current_gesture, samples_collected
    is_collecting = True
    current_gesture = gesture_name
    samples_collected = 0
    set_message(f"Mantenga la seña frente a la cámara. Recolectando '{gesture_name}'...", 3)

In [160]:
def stop_collection():
    """
    Detiene la recolección de datos.
    """
    global is_collecting, current_gesture, samples_collected
    is_collecting = False
    current_gesture = ""
    samples_collected = 0
    set_message("Recolección finalizada", 2)

In [161]:
def save_data():
    """
    Guarda los datos recolectados en disco.
    """
    global data, labels, gesture_data
    data_dict = {
        "features": data, 
        "labels": labels
        }
    with open(f"{data_dir}/{gesture_data}", "wb") as f:
        pickle.dump(data_dict, f)
    set_message(f"Datos guardados: {len(data)} muestras", 1)

In [162]:
def collect_sample(landmarks):
    """
    Recolecta una muestra de landmarks para la seña actual.

    Args:
        landmarks: Lista de landmarks extraídos (126 elementos).

    Returns:
        bool: True si se completó la recolección (max_samples alcanzado), False en caso contrario.
    """
    global data, labels, samples_collected, last_sample_time, is_collecting

    if not is_collecting:
        return False
    
    current_time = time.time()
    # Verificar si ha pasado suficiente tiempo desde la última muestra
    if current_time - last_sample_time >= sample_delay:
        data.append(landmarks)
        labels.append(current_gesture)
        samples_collected += 1
        last_sample_time = current_time

        # Guardar datos periódicamente
        if samples_collected % 10 == 0:
            save_data()
            
        # Verificar si se ha completado la recolección
        if samples_collected >= max_samples:
            stop_collection()
            return True
    return False

In [163]:
def load_data():
    """
    Carga los datos recolectados desde disco.

    Returns:
        bool: True si los datos se cargaron correctamente, False en caso contrario.
    """
    global data, labels, gesture_data
    try:
        with open(f"{data_dir}/{gesture_data}", "rb") as f:
            data_dict = pickle.load(f)
            data = data_dict["features"]
            labels = data_dict["labels"]
        set_message(f"Datos cargados: {len(data)} muestras", 2)
        return True
    except:
        set_message("No se encontraron datos previos", 2)
        return False


In [164]:
def create_neural_network(input_shape, num_classes):
    """
    Crea una red neuronal liviana para reconocimiento de gestos.

    Args:
        input_shape: Dimensión de las características de entrada (int, ej. 126).
        num_classes: Número de clases a predecir (int).

    Returns:
        Modelo de red neuronal compilado (tensorflow.keras.Model).
    """
    model = Sequential([
        # Capa de entrada con regularización para prevenir sobreajuste
        Dense(64, activation='relu', input_shape=(input_shape,), 
              kernel_regularizer=l2(0.001)),
        BatchNormalization(),
        Dropout(0.3), # Dropout para mejorar generalización

        # Capa oculta 
        Dense(32, activation='relu', kernel_regularizer=l2(0.001)),
        BatchNormalization(),
        Dropout(0.2),

        # Capa de salida
        Dense(num_classes, activation='softmax')
    ])

    # Compilar con optimizador Adam y tasa de aprendizaje reducida para estabilidad
    model.compile(optimizer=Adam(learning_rate=0.001), 
                  loss='sparse_categorical_crossentropy', 
                  metrics=['accuracy'])
    return model


In [165]:
def train_model():
    """
    Entrena el modelo de red neuronal con los datos recolectados.

    Returns:
        float: Precisión del modelo en datos de prueba.
    """
    global data, labels, scaler, label_encoder, is_trained, metrics
    if len(data) < 10:
        set_message("Se necesitan más datos para entrenar", 2)
        return 0
    
    set_message("Preparando datos para entrenamiento...", 1)

    # Convertir a arrays NumPy
    X = np.array(data)
    y = np.array(labels)

    # Codificar etiquetas
    y_encoded = label_encoder.fit_transform(y)
    
    # Dividir datos en conjuntos de entrenamiento y prueba
    X_train, X_test, y_train, y_test = train_test_split(
        X, y_encoded, test_size=0.2, random_state=42, stratify=y_encoded
    )
    
    # Normalizar datos
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)
    

    # Crear modelo
    num_classes = len(set(y_encoded))
    set_message(f"Entrenando modelo con {num_classes} clases...", 2)
    
    model = create_neural_network(X_train.shape[1], num_classes)
    
    # Callbacks para mejorar el entrenamiento
    callbacks = [
        EarlyStopping(
            monitor='val_loss', 
            patience=10, 
            restore_best_weights=True),

        ReduceLROnPlateau(
            monitor='val_loss', 
            factor=0.5, 
            patience=5, 
            min_lr=0.0001)
    ]

    # Medir tiempo de entrenamiento
    start_time = time.time()

    # Entrenar el modelo
    history = model.fit(
        X_train, y_train, 
        epochs=50, 
        batch_size=32, 
        validation_split=0.2,
        callbacks=callbacks, 
        verbose=1)
    
    # Calcular tiempo de entrenamiento
    training_time = time.time() - start_time
    
    # Evaluar el modelo
    test_loss, test_accuracy = model.evaluate(X_test, y_test, verbose=0)
    y_pred = np.argmax(model.predict(X_test), axis=-1)
    
    # Guardar métricas
    metrics.update({
        'accuracy': test_accuracy,
        'val_accuracy': max(history.history['val_accuracy']),
        'training_time': training_time,
        'confusion_matrix': confusion_matrix(y_test, y_pred),
        'report': classification_report(y_test, y_pred, target_names=label_encoder.classes_)
    })
    
    # Guardar el modelo y componentes
    model.save(model_file)
    with open(scaler_file, 'wb') as f:
        pickle.dump(scaler, f)
    with open(encoder_file, 'wb') as f:
        pickle.dump(label_encoder, f)
    
    is_trained = True
    set_message(f"Modelo entrenado. Precisión: {test_accuracy:.2f}", 3)
    
    # Imprimir reporte detallado
    print("\n--- Informe del Modelo ---")
    print(f"Precisión en datos de prueba: {test_accuracy:.4f}")
    print(f"Tiempo de entrenamiento: {training_time:.2f} segundos")
    print("\nClasificación detallada:")
    print(metrics['report'])
    
    return test_accuracy

In [166]:
def load_saved_model():
    """
    Carga un modelo previamente entrenado desde disco.

    Returns:
        Modelo cargado (tensorflow.keras.Model) o None si falla la carga.
    """
    global scaler, label_encoder, is_trained
    try:
        # Cargar modelo, scaler y codificador de etiquetas
        model = load_model(model_file)
        #model = tf.keras.models.load_model(model_file)
        with open(scaler_file, 'rb') as f:
            scaler = pickle.load(f)
        with open(encoder_file, 'rb') as f:
            label_encoder = pickle.load(f)
        is_trained = True
        set_message("Modelo de red neuronal cargado correctamente", 2)
        return model
    except Exception as e:
        print(f"Error al cargar el modelo: {str(e)}")
        set_message("No se encontró un modelo guardado", 2)
        return None

In [167]:
def predict_tflite(landmarks, tflite_model, scaler, label_encoder, threshold=0.9):
    """
    Predice la seña a partir de los landmarks usando el modelo TFLite.

    Args:
        landmarks: Lista de landmarks extraídos (126 elementos).
        tflite_model: Instancia de TFLiteModel.
        scaler: Objeto StandardScaler ya entrenado.
        label_encoder: Objeto LabelEncoder ya entrenado.
        threshold: Valor de confianza mínimo para considerar la predicción.

    Returns:
        tuple: Nombre de la seña predicha (str) y confianza (float).
    """
    # Preprocesar datos
    X = np.array([landmarks])
    X_scaled = scaler.transform(X)
    
    # Realizar la predicción con TFLite
    prediction_probs = tflite_model.predict(X_scaled)[0]
    prediction_idx = np.argmax(prediction_probs)
    confidence = prediction_probs[prediction_idx]
    
    # Decodificar la clase
    try:
        prediction_label = label_encoder.inverse_transform([prediction_idx])[0]
    except Exception as e:
        prediction_label = "Desconocido"
    
    # Solo devolver predicción si la confianza es suficiente
    if confidence >= threshold:
        return prediction_label, confidence
    return "Desconocido", confidence


In [168]:
def get_available_gestures():
    """
    Obtiene la lista de gestos disponibles en el conjunto de datos.

    Returns:
        list: Lista de nombres de gestos únicos.
    """
    return list(set(labels))

In [169]:
#NUEVO MENSAJE QUE DICE SI EXISTE EL MDELO
def check_model_exists():
    """Verifica si el archivo del modelo existe."""
    return os.path.exists(model_file)

In [170]:
global model_file, model_tflite

try:
    if not os.path.exists(model_file):
        raise FileNotFoundError(f"El archivo {model_file} no existe.")

    # Cargar el modelo entrenado
    modelo = load_model(model_file)

    # Convertir a TensorFlow Lite
    converter = tf.lite.TFLiteConverter.from_keras_model(modelo)
    tflite_model = converter.convert()

    # Guardar el modelo convertido
    with open(model_tflite, "wb") as f:
        f.write(tflite_model)
        
    print("Modelo convertido a TensorFlow Lite.")

except Exception as e:
    print("Error al cargar o convertir el modelo:", e)


Error al cargar o convertir el modelo: El archivo hand_gesture_nn_model_v13_2.h5 no existe.


In [171]:
import os

model_path = "modelo_optimizadotl.tflite"
if not os.path.exists(model_path):
    print(f"El archivo {model_path} no existe. Asegúrate de haberlo generado correctamente.")
else:
    tflite_model = TFLiteModel(model_path)

In [172]:
def print_menu():
    print("\n=== MENU PRINCIPAL ===")
    print("1. Recolectar nueva seña")
    print("2. Entrenar modelo")
    print("3. Listar señas cargadas")
    print("4. Evaluar en tiempo real")
    print("5. Salir")

In [173]:
def list_gestures():
    # Asumiendo que 'labels' es la lista donde se guardan las señas
    if not labels:
        print("No hay señas guardadas.")
    else:
        unique_gestures = list(set(labels))
        print("\n--- Señas Guardadas ---")
        for i, gesture in enumerate(unique_gestures, 1):
            print(f"{i}. {gesture}")

In [174]:
def run_collection_mode():
    # Inicia la cámara
    try:
        cap = UDPCamera()
        print("Cámara UDP iniciada para recolección.")
    except Exception as e:
        print(f"Error al iniciar la cámara: {str(e)}")
        return
    
    

    while is_collecting:  # Asumiendo que 'is_collecting' se activa en start_collection()
        ret, frame = cap.read()
        if not ret:
            time.sleep(0.1)
            continue

        landmarks, hands_detected = extract_hand_landmarks(frame)

        frame_h, frame_w, _ = frame.shape

        # Mostrar información en pantalla durante la recolección
        progress = int((samples_collected / max_samples) * frame_w)
        cv2.rectangle(frame, (0, 0), (progress, 20), (0, 255, 0), -1)
        cv2.putText(frame, f"Recolectando: {current_gesture} ({samples_collected}/{max_samples})", 
                    (10, 50), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
        
        if not hands_detected:
            cv2.putText(frame, "¡Muestra las manos!", (frame_w//2 - 100, frame_h//2), 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
        
        cv2.imshow("Recolectar Señas", frame)
        
        key = cv2.waitKey(1)
        # Puedes agregar una tecla para finalizar la recolección, por ejemplo 'm' para volver al menú.
        if key == ord('m'):
            break
    
    cap.release()
    cv2.destroyAllWindows()
    save_data()  # Guarda los datos recolectados


In [175]:
def run_evaluation_mode():
    global model_tflite
    # Inicializa el modelo TFLite si aún no se ha cargado
    if os.path.exists(model_tflite):
        tflite_model = TFLiteModel(model_tflite)
    else:
        print("El modelo TFLite no existe. Conviértelo primero.")
        return

    # Inicia la cámara
    try:
        cap = UDPCamera()
        print("Cámara UDP iniciada para evaluación en tiempo real.")
    except Exception as e:
        print(f"Error al iniciar la cámara: {str(e)}")
        return
    
    while True:
        ret, frame = cap.read()
        if not ret:
            time.sleep(0.1)
            continue

        landmarks, hands_detected = extract_hand_landmarks(frame)
        frame_h, frame_w, _ = frame.shape

        if hands_detected:
            prediction, confidence = predict_tflite(landmarks, tflite_model, scaler, label_encoder, threshold=0.9)
            color = (0, 255, 0) if confidence > 0.9 else (0, 165, 255)
            cv2.putText(frame, f"Seña: {prediction}", (10, 50), 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.9, color, 2)
            cv2.putText(frame, f"Confianza: {confidence:.2%}", (10, 90), 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.7, color, 2)
            
            # Extraer valor escalar en caso de que 'confidence' sea un array
            confidence_value = np.max(confidence) if isinstance(confidence, np.ndarray) else confidence

            if confidence_value > 0.99 and prediction != "Desconocido":
                threading.Thread(target=speak_text, args=(prediction,), daemon=True).start()
        else:
            cv2.putText(frame, "Acerca las manos a la cámara", (frame_w//4, frame_h//2), 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 0), 2)
        
        cv2.putText(frame, "Presiona M para volver al menú", (10, frame_h - 30), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 0), 1)
        cv2.imshow("Evaluación en Tiempo Real", frame)
        
        key = cv2.waitKey(1)
        if key == ord('m'):
            break

    cap.release()
    cv2.destroyAllWindows()


In [176]:
def main():
    global model, is_trained, data, labels

    # Inicialización del sistema
    is_trained = False
    model = None
    data = []
    labels = []
    
    # Cargar datos existentes
    load_data()
    
    # Intentar cargar modelo si existe
    if check_model_exists():
        model = load_saved_model()
        is_trained = True
    else:
        is_trained = False

    # Mostrar el menú en la consola
    print_menu()

    # Bucle principal de selección en consola
    while True:
        opcion = input("\nSelecciona una opción (1, 2, 3, 4 o 5): ").strip()
        
        if opcion == '1':
            # Recolección de señas
            gesture_name = input("Ingrese nombre de la seña (ej. 'Hola'): ")
            if gesture_name:
                start_collection(gesture_name)
                # Iniciar la cámara para mostrar video durante la recolección
                run_collection_mode()
                
        elif opcion == '2':
            if len(data) > 10:
                train_model()
                model = load_saved_model() if check_model_exists() else None
                is_trained = True
                print("Entrenamiento completado. Modelo entrenado.")
            else:
                print("¡Necesitas al menos 10 muestras para entrenar!")
                
        elif opcion == '3':
            list_gestures()  # Lista las señas cargadas

        elif opcion == '4':
            if is_trained:
                # Inicializar modo evaluación en tiempo real
                print("Modo de evaluación activado.")
                run_evaluation_mode()
            else:
                print("¡Entrena el modelo primero (Opción 2)!")
                
        elif opcion == '5':
            print("Saliendo del programa...")
            break
        else:
            print("Opción inválida, intenta nuevamente.")
        
        # Mostrar nuevamente el menú luego de finalizar la opción seleccionada.
        print_menu()


In [177]:
if __name__ == "__main__":
    #run()
    main()


=== MENU PRINCIPAL ===
1. Recolectar nueva seña
2. Entrenar modelo
3. Listar señas cargadas
4. Evaluar en tiempo real
5. Salir
Error al iniciar la cámara: [WinError 10048] Solo se permite un uso de cada dirección de socket (protocolo/dirección de red/puerto)
Cámara UDP liberada

=== MENU PRINCIPAL ===
1. Recolectar nueva seña
2. Entrenar modelo
3. Listar señas cargadas
4. Evaluar en tiempo real
5. Salir
Saliendo del programa...
